<a href="https://colab.research.google.com/github/Vigneshthanga/SpecialTopics/blob/master/Assignment-3/resnet_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries

In [44]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import matplotlib
import matplotlib.pyplot as plt

## Download flower dataset

In [45]:
data_dir = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)

In [46]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [47]:
BATCH_SIZE = 64

In [48]:
IMG_SIZE = (224, 224)
train_dataset = image_dataset_from_directory(data_dir,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)

Found 3670 files belonging to 5 classes.


## Defining datagen and dataflow

In [49]:
datagen_kwargs = dict(rescale=1./255, validation_split=.20)
dataflow_kwargs = dict(target_size=IMG_SIZE, batch_size=BATCH_SIZE,
                   interpolation="bilinear")

## Valid datagen

In [50]:
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    **datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
    data_dir, subset="validation", shuffle=False, **dataflow_kwargs)

Found 731 images belonging to 5 classes.


## Train datagen 

In [51]:
train_datagen = valid_datagen
train_generator = train_datagen.flow_from_directory(
    data_dir, subset="training", shuffle=True, **dataflow_kwargs)

Found 2939 images belonging to 5 classes.


In [52]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')


In [53]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(64, 7, 7, 2048)


In [54]:
base_model.trainable = False

## Adding custom layers

In [55]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(64, 2048)


In [56]:
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(64, 1)


## Defining new model and on top of base model

In [57]:
inputs = tf.keras.Input(shape=(224, 224, 3))
x = tf.keras.applications.resnet.preprocess_input(inputs)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [58]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [59]:
initial_epochs = 10
loss0, accuracy0 = model.evaluate(valid_generator)

12/12 [==============================] - 3s 281ms/step - loss: 0.6780 - accuracy: 0.8000


## Train the model freezing the base layers

In [60]:
history = model.fit(train_generator,
                    epochs=initial_epochs,
                    validation_data=valid_generator)

Epoch 1/10
46/46 [==============================] - 20s 436ms/step - loss: 0.6062 - accuracy: 0.7880 - val_loss: 0.5276 - val_accuracy: 0.8000
Epoch 2/10
46/46 [==============================] - 19s 417ms/step - loss: 0.5268 - accuracy: 0.7998 - val_loss: 0.5022 - val_accuracy: 0.8000
Epoch 3/10
46/46 [==============================] - 19s 418ms/step - loss: 0.5153 - accuracy: 0.8000 - val_loss: 0.5007 - val_accuracy: 0.8000
Epoch 4/10
46/46 [==============================] - 19s 423ms/step - loss: 0.5139 - accuracy: 0.8000 - val_loss: 0.5008 - val_accuracy: 0.8000
Epoch 5/10
46/46 [==============================] - 19s 423ms/step - loss: 0.5132 - accuracy: 0.8000 - val_loss: 0.5008 - val_accuracy: 0.8000
Epoch 6/10
46/46 [==============================] - 19s 421ms/step - loss: 0.5139 - accuracy: 0.8000 - val_loss: 0.5008 - val_accuracy: 0.8000
Epoch 7/10
46/46 [==============================] - 19s 412ms/step - loss: 0.5129 - accuracy: 0.8000 - val_loss: 0.5008 - val_accuracy: 0.8000

## Unfreeze the base models

In [61]:
base_model.trainable = True

In [62]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

Number of layers in the base model:  175


In [63]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])


## Training after unfreezing

In [64]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_generator,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=valid_generator)

Epoch 10/20
46/46 [==============================] - 23s 501ms/step - loss: 0.5091 - accuracy: 0.8000 - val_loss: 0.5018 - val_accuracy: 0.8000
Epoch 11/20
46/46 [==============================] - 21s 467ms/step - loss: 0.5029 - accuracy: 0.8000 - val_loss: 0.5012 - val_accuracy: 0.8000
Epoch 12/20
46/46 [==============================] - 22s 467ms/step - loss: 0.5019 - accuracy: 0.8000 - val_loss: 0.5004 - val_accuracy: 0.8000
Epoch 13/20
46/46 [==============================] - 22s 470ms/step - loss: 0.5015 - accuracy: 0.8000 - val_loss: 0.5014 - val_accuracy: 0.8000
Epoch 14/20
46/46 [==============================] - 22s 475ms/step - loss: 0.5014 - accuracy: 0.8000 - val_loss: 0.5007 - val_accuracy: 0.8000
Epoch 15/20
46/46 [==============================] - 22s 473ms/step - loss: 0.5012 - accuracy: 0.8000 - val_loss: 0.5004 - val_accuracy: 0.8000
Epoch 16/20
46/46 [==============================] - 22s 469ms/step - loss: 0.5012 - accuracy: 0.8000 - val_loss: 0.5007 - val_accuracy:

## Comparision between SIMCLR v2 and Resnet: Thus we can see the resnet models is easy to train and it achieves good accuracy sooner but it takes too many epochs to converge. But, SIM CLR v2 didn't achieve good accuracy initially but converges sooner than resent.